# BERT-Base-Chinese-Finetuning-Financial-News-Sentiment-V2 Model

This part is the same as task 1, question 1, because I already completed the Sentiment Polarity Analysis in question 1 to save time.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd

# Load the pretrained BERT model and tokenizer
model_name = "hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Check CUDA availability and move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Load news data
result_news_data_with_sentiment = pd.read_excel('result_df.xlsx')
# result_news_data = pd.read_excel('result_news_data.xlsx')

# Perform sentiment analysis
sentiment_analysis = lambda text: (
    int(torch.nn.functional.softmax(
        model(**tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)).logits, dim=-1
    )[:, 2].item() > torch.nn.functional.softmax(
        model(**tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)).logits, dim=-1
    )[:, 0].item())
)

# Apply sentiment analysis to the 'NewsContent' column and add the results as a new 'label' column
result_news_data_with_sentiment['label'] = [sentiment_analysis(content) for content in result_news_data['NewsContent']]


In [ ]:
result_news_data_with_sentiment

,NewsID,NewsContent,Explicit_Company,label
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,建设银行,0
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,农业银行,1
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,"外运发展, 中国国航",1
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,胜利股份,1
4,8,由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致...,冠农股份,1
...,...,...,...,...
187293,1037007,10月13日，今日共有43只涨停股，5只跌停股。其中，涨停股主要集中在华为概念股、减肥药概念...,"模塑科技, 龙版传媒, 莎普爱思, 光洋股份, 通化金马, 圣龙股份, 通宇通讯, 欧菲光",0
187294,1037009,吉电股份10月13日在交易所互动平台中披露，截至10月10日公司股东户数为171303户，较...,吉电股份,0
187295,1037025,10月12日晚间，三星医疗发布2023年前三季度业绩预告，公司预计前三季度实现归属于母公司所...,三星医疗,1
187296,1037030,每经AI快讯，有投资者在投资者互动平台提问：公司领导，请问公司经营是不是出现重大问题了，股票...,亿华通,0


In [12]:
# Save the modified DataFrame to a new XLSX file
result_news_data_with_sentiment.to_excel('Task1.xlsx', index=False)